# Singular Value Decomposition



# Toy example: Artificial Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook
plt.rcParams['figure.dpi'] = 150

# pca takes centered data matrix
def pca(data):
    covariance_matrix = (data.transpose() @ data) / data.shape[0]
    
    eigvals, eigvecs = np.linalg.eig(covariance_matrix)
    
    sorted_idx = np.argsort(eigvals)[::-1]
    
    return eigvals[sorted_idx], eigvecs[:,sorted_idx]

In [ ]:
# Number of data points
N = 100

# Data dimension
D = 3

# Generate data
data = np.random.normal(size=(N, D))
center = np.mean(data, axis=0)
data = data - center

eigvals, eigvecs = pca(data)
# Eigenvalues are real and positive
print(eigvals)

# Eigenvectors are orthogonal
products = eigvecs.transpose() @ eigvecs
print(products)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)

ax.scatter(data[:, 0], data[:, 1], data[:, 2])
for v in np.transpose(eigvecs):
    ax.plot3D([0, v[0]*3], [0, v[1]*3], [0, v[2]*3])

In [ ]:
# Number of data points
N = 100

# Random Embedding Matrix
embedding = np.random.normal(size=(2, 3))

# Generate and center data
data = np.random.normal(size=(N, 2)) @ embedding + np.random.normal(size=(N, 3))*0.1
center = np.mean(data, axis=0)
data = data - center

eigvals, eigvecs = pca(data)
print(f'Eigenvalues are {eigvals}')

(U, singvals, V) = np.linalg.svd(data)
print(f'Singular Values are {singvals}')
print(f'Squared Singular Values are {singvals*singvals}')


In [ ]:
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)

ax.scatter(data[:, 0], data[:, 1], data[:, 2])
for v in np.transpose(eigvecs):
    ax.plot3D([0, v[0]*3], [0, v[1]*3], [0, v[2]*3])

In [ ]:
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)

ax.scatter(data[:, 0], data[:, 1], data[:, 2])
for v in np.transpose(eigvecs):
    ax.plot3D([0, v[0]*3], [0, v[1]*3], [0, v[2]*3], 'b')
for v in np.transpose(V):
    ax.plot3D([0, v[0]*3], [0, v[1]*3], [0, v[2]*3], 'r')


In [ ]:
# Transform data into eigenvectors coordinate system
transformed_data = data @ eigvecs
print(f'Variance along first eigenvector: {np.var(transformed_data[:, 0])}')
print(f'Variance along second eigenvector: {np.var(transformed_data[:, 1])}')
print(f'Veriance along third eigenvector: {np.var(transformed_data[:, 2])}')

print(f'Eigenvalues: {eigvals}')

transformed_data = data @ V
print(f'Variance along first eigenvector: {np.var(transformed_data[:, 0])}')
print(f'Variance along second eigenvector: {np.var(transformed_data[:, 1])}')
print(f'Veriance along third eigenvector: {np.var(transformed_data[:, 2])}')

print(f'Squared Singular values: {singvals*singvals}')

In [ ]:
# Neglect the dimension which corresponds to the lowest eigenvalue
transformed_data[:, 2] = 0.

# Transform to the original coordinate system
transformed_back = transformed_data @ eigvecs.transpose()

average_squared_distance = np.sum((data - transformed_back)**2) / N
print(f'Average Squared Distance from original data = {average_squared_distance}')

# EigenFaces

In [ ]:
import pandas as pd

df = pd.read_csv('face_data.csv')

# 400 Samples, 64x64 images in shades of gray + 1 target label = 4097
df.shape

In [ ]:
def plot_faces(pixels):
    fig, axes = plt.subplots(5, 5, figsize=(6, 6))
    for i, ax in enumerate(axes.flat):
        ax.imshow(np.array(pixels)[i].reshape(64, 64), cmap='gray')
    plt.show()
    
X = df.drop('target', axis=1)
y = df['target']

plot_faces(X)

In [ ]:
# Bring the data to the format for our pca function
data = np.array(X)

data.shape

In [ ]:
# Compute SVD.
U, singvals, V = np.linalg.svd(data)

fig = plt.figure()
plt.plot(singvals[:50])
plt.show()

In [ ]:
# Plot eigenfaces
plot_faces(V)

In [ ]:
transformed_data = data @ V[:25, :].T

print(f'Shape of the transformed data is {transformed_data.shape}')

transformed_back = transformed_data @ V[:25, :]

plot_faces(transformed_back)

# Image Compression

In [ ]:
import matplotlib.image as mpimg

img = mpimg.imread('cat.png')

print(img.shape) 

fig = plt.figure()
plt.imshow(img)

In [ ]:
# Treat every column as a vector
data = np.reshape(img, (900, 900*4)) 
print(data.shape) 

# Center the data
center = np.mean(data, axis=0)

centered_data = data - center

# Compute SVD
U, singvals, V = np.linalg.svd(data)

fig = plt.figure()
plt.plot(singvals[:50])
plt.show()

In [ ]:
n = 50
# Project to first n eigenvectors
transformed_data = data @ V[:n, :].T

print(f'Shape of the transformed data is {transformed_data.shape}')

transformed_back = transformed_data @ V[:n, :] 

# reshape the transformed back data
rec_img = np.reshape(transformed_back, (900, 900, 4))

fig = plt.figure()
plt.imshow(rec_img)

In [ ]:
from skimage.transform import resize
from skimage.color import rgb2gray, rgba2rgb

img_resized = resize(img, (64, 64))
img_bw = rgb2gray(rgba2rgb(img_resized))

fig = plt.figure()

plt.imshow(img_bw, cmap='gray')

# Compressing a cat with EigenFaces?

In [ ]:
# Bring the data to the format from eigenfaces 
data = np.array(X)

# Compute SVD.
U, singvals, V = np.linalg.svd(data)

# Pretend the cat is a face
cat = np.reshape(img_bw, (1, 4096))


In [ ]:
data = np.reshape(img_bw, (1, 4096))

fig, axes = plt.subplots(4, 4, figsize=(6, 6))
for i, ax in enumerate(axes.flat):
    num_vectors = 50 * (i + 1)
    reconstruction = data @ V[:num_vectors, :].T @ V[:num_vectors, :]
    ax.imshow(reconstruction.reshape(64, 64), cmap='gray')
    ax.set_title(f"#{num_vectors} vectors")
plt.show()


# Solving differential equations

## Linear system

In [ ]:
from scipy.integrate import odeint

In [ ]:
a,b,c,d=1, 0,0,1 # Identity
# a,b,c,d=-4,1,2,3 # Compression - expansion
# a,b,c,d=1,-1,1,1 # Complex eigenvalues
# a,b,c,d=1,1,0,1 # Non-diagonalizable
transformation = np.array([[a, b], [c, d]])
print("Coefficients:", transformation)

(vals, vecs) = np.linalg.eig(transformation)
print("Eigenvalues:",vals )
print("Eigenvectors:",vecs[:, 0], vecs[:, 1] )

# Solving the ODE
def dX_dt(X, t):
    return [a * X[0] + b * X[1], c * X[0] + d * X[1]]

ts = np.linspace(0, 0.7, 100)

fig = plt.figure()
for px, py in circle.T:
    Ps = odeint(dX_dt, [px, py], ts)
    plt.plot(Ps[:,0],  Ps[:,1])

r = 5
plt.arrow(0,0,r * vals[0] * vecs[0,0], r * vals[0] * vecs[1,0],head_width=1,head_length=2)
plt.arrow(0,0,r * vals[1] * vecs[0,1], r * vals[1] * vecs[1,1],head_width=1,head_length=2)
    
plt.show()

In [ ]:
# Analytical solution

def time(P, t):
    return vecs @  np.diag(np.exp(vals * t)) @ np.linalg.inv(vecs) @ P

fig = plt.figure()
for P in circle.T:
    Ps = np.array([time(P, t) for t in ts])
    plt.plot(Ps[:,0],  Ps[:,1])

r = 5
plt.arrow(0,0,r * vals[0] * vecs[0,0], r * vals[0] * vecs[1,0],head_width=1,head_length=2)
plt.arrow(0,0,r * vals[1] * vecs[0,1], r * vals[1] * vecs[1,1],head_width=1,head_length=2)
    
plt.show()

In [ ]:
# Analytic solution, second order, for Non-diagonalizable
# Only for a,b,c,d=1,1,0,1 # Non-diagonalizable

w = np.array([0,1])

assert ((transformation - vals[0] * np.identity(2)) @ w == vecs[:, 0]).all()

def time(P, t):
    return [P[0] * np.exp(t) + P[1] * t * np.exp(t) , P[1] * np.exp(t)]

ts = np.linspace(0, 0.7, 100)

fig = plt.figure()
for P in circle.T:
    Ps = np.array([time(P, t) for t in ts])
    plt.plot(Ps[:,0],  Ps[:,1])

r = 5
plt.arrow(0,0,r * vals[0] * vecs[0,0], r * vals[0] * vecs[1,0],head_width=1,head_length=2)
plt.arrow(0,0,r * vals[1] * vecs[0,1], r * vals[1] * vecs[1,1],head_width=1,head_length=2)
    
plt.show()

In [ ]:
np.array([vecs[:, 0], w])

# Eigendecomposition and Quantum Mechanics

## Eigendecomposition of a continuous system


$$ H = - \frac{\hbar^2}{2m} \nabla^2 + V(x) $$ 

Trick to solve for $\nabla^2$

$$\frac{\partial^2}{\partial x} \psi(x) \simeq \frac 1 {dx^2} \left(\psi(x+dx) - 2*\psi(x) + \psi(x-dx)  \right) $$

In [ ]:
n = 1000
xmax = 10
x = np.linspace(-xmax,xmax,num=n)
dx = 2*xmax/(n-1)

# Potential
V = np.diag(x*x) / 2
# Kinetic term
T =  -2 * np.identity(n) + np.diag([1]*(n-1), k=1) + np.diag([1]*(n-1), k=-1)
T =  - T / 2 / dx**2 
# Hamiltonian
H = T + V 

(vals, vecs) = np.linalg.eigh(H)

sorted(vals.round(3))[:5]

In [ ]:
fig = plt.figure()
plt.plot(x, np.diag(V))
for i in range(5):
    plt.plot(x, vals[i] + 5 * vecs[:, i])

plt.ylim([0, 5])
plt.show()

## Eigenbases of a Discrete hamiltonian

In [ ]:
def eigs(interaction):
    H = np.array([[interaction,1,1,0],[1,0,0,1],[1,0,0,1],[0,1,1,0]])
    (vals, vecs) = np.linalg.eigh(H)
    return vals
    
deltas = range(10)
l = np.array([eigs(delta) for delta in deltas])
    
fig = plt.figure()
plt.plot(deltas, l)
plt.show()